In [1]:
import os
import numpy as np

data = 'archive/Garbage classification/Garbage classification'

print(os.listdir(data))
from PIL import Image

png_filepath = data + '/paper/paper312.jpg'

['paper', '.DS_Store', 'metal', 'cardboard', 'trash', 'glass', 'plastic']


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 2000 # Make this number of images or do actual BGD
target_size = (128, 128) # Make this smaller

train = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,validation_split=0.1,rescale=1./255,
                         shear_range = 0.1,zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

test =ImageDataGenerator(rescale=1./255,validation_split=0.1)

train_generator=train.flow_from_directory(data,target_size=target_size,batch_size=batch_size,class_mode='sparse',subset='training')

test_generator=test.flow_from_directory(data,target_size=target_size,batch_size=batch_size,class_mode='categorical',subset='validation')

2023-11-09 11:12:03.668830: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models, layers

labels = list(train_generator.class_indices.keys())
print(labels)
num_classes = len(labels)

# Build Model
model = models.Sequential()


model.add(layers.Conv2D(8,(3,3), padding='same', input_shape=(128, 128,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2))) 

model.add(layers.Conv2D(8,(3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2))) 

model.add(layers.Conv2D(16,(3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(16,(3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2))) 

model.add(layers.Conv2D(32,(3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(32,(3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2))) 


model.add(layers.Flatten())
model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(num_classes,activation='softmax'))

# adding a dropout layer, batch normalization, early stopping

model.summary()

In [ ]:
import tensorflow as tf
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.CategoricalCrossentropy())

In [ ]:
model.fit(train_generator, epochs=100, validation_data=(test_generator))